In [1]:
# Model packages
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Reshape, Activation, LSTM, TimeDistributed, BatchNormalization, Dropout

# Dataset management packages
from spivutils.synthetic_datasets.spid import load_data
from spivutils.batch_generators.keras_generator import batch_data
from spivutils.common_tools.operations import normalization, vectoraddition, thresholding

# General purpose packages
import numpy as np
import gc

Hardware Settings

In [2]:
cpu_devices = tf.config.list_physical_devices('CPU')
gpu_devices = tf.config.list_physical_devices('GPU')

In [ ]:
# Force CPU usage
tf.config.set_visible_devices([], 'GPU')

Model Hyperparameters

In [3]:
chunk_size = 100                                        # Slice  of the dataset
batch_size = 16                                         # Number of patterns shown to the network before weight matrix update
epochs = 50                                             # Number of times that the model sees the dataset
activation = 'relu', 'linear'                           #
optimizer = 'adam'                                      # Weight update after each iteration

learning_rate = 0.001                                           
dropout = 0.5

dense_units = [32, 64, 128, 256, 512, 1024, 2048]       # Hidden units per layer
lstm_units = [32, 64, 128, 256, 512, 1024]              # Hidden units per layer

filters = [32, 64, 128, 256, 512]
kernel_size = [(3,3), (5,5), (7,7), (9,9), (11,11)]
pool_size = [(2,2), (4,4), (6,6), (8,8), (10,10)]
padding = 'same'

Data management

In [4]:
# Data importing
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data()

# Collect a chunk of the dataset
train_x_chunk = train_x#[0:chunk_size]
train_y_chunk = train_y#[0:chunk_size]
valid_x_chunk = valid_x#[0:chunk_size]
valid_y_chunk = valid_y#[0:chunk_size]

# Load data in batches to avoid memory overload
train_batch = batch_data(train_x_chunk, train_y_chunk, batch_size)
valid_batch = batch_data(valid_x_chunk, valid_y_chunk, batch_size)


 Checking files ...

 train_x.npy  Found!

 train_y.npy  Found!

 valid_x.npy  Found!

 valid_y.npy  Found!

 test_x.npy  Found!

 test_y.npy  Found!

 Importing data ...

 Data loading successful!


Pre-processing

In [5]:
# Add pre-processing operations
train_batch.add_x_preprocessing_operation(thresholding)
train_batch.add_x_preprocessing_operation(normalization)
train_batch.add_y_preprocessing_operation(vectoraddition)

valid_batch.add_x_preprocessing_operation(thresholding)
valid_batch.add_x_preprocessing_operation(normalization)
valid_batch.add_y_preprocessing_operation(vectoraddition)

Model

In [6]:
input_shape = train_batch[0][0][0,].shape
output_shape = train_batch[0][1][0,].shape

In [7]:
def build_convnet(input_shape, filters, kernel_size, pool_size, activation, padding):

    model = Sequential()

    model.add(Conv2D(filters = filters[0], kernel_size = kernel_size[4], padding = padding, input_shape = input_shape))
    model.add(Activation(activation[0]))
    model.add(MaxPooling2D(pool_size = pool_size[4]))

    model.add(Conv2D(filters = filters[1], kernel_size = kernel_size[3], padding = padding))
    model.add(Activation(activation[0]))
    model.add(MaxPooling2D(pool_size = pool_size[3]))

    model.add(Conv2D(filters = filters[2], kernel_size = kernel_size[2], padding = padding))
    model.add(Activation(activation[0]))
    model.add(MaxPooling2D(pool_size = pool_size[2]))

    model.add(Flatten())

    return model

In [8]:
def build_model(input_shape, output_shape, filters, kernel_size, pool_size, lstm_units, dense_units, activation, padding):

    convnet = build_convnet(input_shape[1:], filters, kernel_size, pool_size, activation, padding)

    model = Sequential()

    model.add(TimeDistributed(convnet, input_shape = input_shape))

    model.add(LSTM(units = lstm_units[0], return_sequences = False))

    model.add(Dense(units = dense_units[6]))
    model.add(Activation('relu'))

    model.add(Dense(units = dense_units[5]))
    model.add(Activation('relu'))

    model.add(Dense(units = dense_units[4]))
    model.add(Activation('relu'))

    model.add(Dense(units = dense_units[3]))
    model.add(Activation('relu'))

    model.add(Dense(units = dense_units[2]))
    model.add(Activation('relu'))

    model.add(Dense(units = dense_units[1]))
    model.add(Activation('relu'))

    model.add(Dense(units = dense_units[0]))
    model.add(Activation('relu'))

    model.add(Dense(np.prod(output_shape)))
    model.add(Activation('linear'))

    model.add(Reshape(output_shape))

    return model

In [9]:
model = build_model(input_shape, output_shape, filters, kernel_size, pool_size, lstm_units, dense_units, activation, padding)

model.compile(loss = 'mean_squared_error', optimizer = 'adam')

In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 2, 128)           571392    
 ibuted)                                                         
                                                                 
 lstm (LSTM)                 (None, 32)                20608     
                                                                 
 dense (Dense)               (None, 2048)              67584     
                                                                 
 activation_3 (Activation)   (None, 2048)              0         
                                                                 
 dense_1 (Dense)             (None, 1024)              2098176   
                                                                 
 activation_4 (Activation)   (None, 1024)              0         
                                                      

In [11]:
class CustomCall(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()

In [12]:
history = model.fit(train_batch, validation_data = valid_batch, epochs = epochs, callbacks = [CustomCall()])

Epoch 1/50
945/945 [==============================] - 1085s 1s/step - loss: 5.1514 - val_loss: 4.8911
Epoch 2/50
945/945 [==============================] - 1020s 1s/step - loss: 4.9642 - val_loss: 5.3951
Epoch 3/50
945/945 [==============================] - 1022s 1s/step - loss: 4.9387 - val_loss: 5.2715
Epoch 4/50
945/945 [==============================] - 1011s 1s/step - loss: 4.8792 - val_loss: 5.0312
Epoch 5/50
945/945 [==============================] - 1005s 1s/step - loss: 4.8623 - val_loss: 4.8401
Epoch 6/50
945/945 [==============================] - 1025s 1s/step - loss: 4.8557 - val_loss: 4.9103
Epoch 7/50
945/945 [==============================] - 1017s 1s/step - loss: 4.8472 - val_loss: 4.8555
Epoch 8/50
945/945 [==============================] - 1004s 1s/step - loss: 4.8577 - val_loss: 4.8380
Epoch 9/50
945/945 [==============================] - 1012s 1s/step - loss: 4.8289 - val_loss: 4.8323
Epoch 10/50
945/945 [==============================] - 1056s 1s/step - loss: 4.819

ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/sequential_1/time_distributed/sequential/max_pooling2d/MaxPool/MaxPoolGrad' defined at (most recent call last):
    File "C:\Users\michel.machado\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\michel.machado\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "d:\Pessoal\Velocimetria\vel_env\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "d:\Pessoal\Velocimetria\vel_env\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "d:\Pessoal\Velocimetria\vel_env\lib\site-packages\ipykernel\kernelapp.py", line 725, in start
      self.io_loop.start()
    File "d:\Pessoal\Velocimetria\vel_env\lib\site-packages\tornado\platform\asyncio.p

In [13]:
import matplotlib.pyplot as plt

In [14]:
def plot_loss(history):
    loss = history.history['loss']
    epochs = range(len(loss))
    plt.plot(epochs, loss, 'r')
    plt.title('Training loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')

In [15]:
plot_loss(history)

NameError: name 'history' is not defined

In [ ]:
history.history.keys()